In [21]:
import numpy as np
from __future__ import division
f=open('/home/lkit/wslap/data/A370/image_region_5600scale.dat')
l=np.asarray([map(float,line.split()) for line in f])
f.close()
cx=(l[:,0]+l[:,1])/2.
cy=(l[:,2]+l[:,3])/2.
print cx[1]
print cy

f=open('/home/lkit/wslap/data/A370/image_center_5600scale.dat','w')
for i in range(len(cx)):
    f.write('%.1f\t\t%.1f\t\t%.1f\n' %(cx[i],cy[i],l[i,4]))
f.close()

2860.5
[ 3122.   3049.5  3019.5  2138.   2037.5  2062.5  2072.   2102.   4183.
  4349.   4222.5  3081.5  3094.5  2894.5  1558.5  1536.   1559.5  2990.5
  2975.   2856.   2599.   2555.   1967.   2644.   2888.5  3388.5  3868.5
  3409.5  3060.5  3093.   2897.5  4025.5  2510.5  3919.   3345.   3734.
  4247.5  3206.   2693.   3677.   3630.5  4486.   3965.5  2859.5  2870.
  2784.5  2280.   2779.   2276.5  1525.5  1654.5  4188.   2168.   3774.5
  3756.   4072.   3988.   3005.5  2960.   2876.   4148.5]


In [73]:
from operator import itemgetter, attrgetter
buffer=12
with file('/home/lkit/wslap/data/A370/image_center_5600scale.dat') as f:
    center=np.asarray([map(float,line.split()) for line in f])
cx=center[:,0]; cy=center[:,1];
with file('/home/lkit/Data/Redshift_Catalog_5600scale') as f:
    for i in range(31): next(f)
    catalog=np.asarray([map(float,(line.split())[0:6]) for line in f])
imx=catalog[:,1]; imy=catalog[:,2]
id_list=[]; z=[]; zq=[]
for i in range(len(catalog)):
    for j in range(len(center)):
        if (abs(cx[j]-imx[i])<=buffer and abs(cy[j]-imy[i])<=buffer):
            id_list.append(center[j,2])
            z.append(catalog[i,3])
            zq.append(catalog[i,4])

re=np.asarray([[x,y,z] for x,y,z in zip(id_list,z,zq)])
zout=np.asarray(sorted(re, key=itemgetter(0)))

s1='# id  redshift  quailty\n'
with file('/home/lkit/Data/GLASS/redshift','w') as f:
    f.write(s1)
    np.savetxt(f,zout,fmt='%.2f',delimiter='\t')

In [86]:
from operator import itemgetter, attrgetter
buffer=15
with file('/home/lkit/wslap/data/A370/image_center_5600scale.dat') as f:
    center=np.asarray([map(float,line.split()) for line in f])
cx=center[:,0]; cy=center[:,1];
with file('/home/lkit/Data/Source_Catalog_5600scale') as f:
    for i in range(21): next(f)
    catalog=np.asarray([map(float,line.split()) for line in f])
imx=catalog[:,1]; imy=catalog[:,2]
id_list=[]; flux_auto=[]; fluxerr_auto=[]; ra=[]; dec=[]
for i in range(len(catalog)):
    for j in range(len(center)):
        if (abs(cx[j]-imx[i])<=buffer and abs(cy[j]-imy[i])<=buffer):
            id_list.append(center[j,2])
            flux_auto.append(catalog[i,25])
            fluxerr_auto.append(catalog[i,26])
            ra.append(catalog[i,3])
            dec.append(catalog[i,4]+0.0000803)

re=np.asarray([[x,y,z,z1,z2] for x,y,z,z1,z2 in zip(id_list,flux_auto,fluxerr_auto,ra,dec)])
zout=np.asarray(sorted(re, key=itemgetter(0)))

s1='# id  MAG_AUTO  MAGERR_AUTO     RA     DEC\n'
with file('/home/lkit/Data/GLASS/flux','w') as f:
    f.write(s1)
    np.savetxt(f,zout,fmt='%.6f',delimiter='\t')

In [429]:
from __future__ import division
import itertools
from math import *
def calc(x):
    return 10**((26.46-x)/2.5)
def fratio(x,y,dx,dy):
    return (x/y)*sqrt((dx/x)**2+(dy/y)**2)
def dex(x):
    return x*10-int(x)*10
def decomp(x):
    name=[]
    for i in x:
        for j in i:
            name.append(j)
    return name
with file('/home/lkit/Data/GLASS/flux') as f:
    next(f)
    l=np.asarray([map(float,line.split()) for line in f])
idd=l[:,0]; mag=l[:,1]; magerr=l[:,2]
flux=np.asarray(map(lambda x: 10**((26.46-x)/2.5), mag))    # mag -> flux
fluxerr=[0.5*abs(calc(i[0]+i[1])-calc(i[0]-i[1])) for i in zip(mag,magerr)]
data=np.asarray(zip(idd,flux,fluxerr))    #total combination
print data

####### 1.1/1.2, 1.2/1.3, 1.1/1.3 ###################
# dig=set(map(lambda x: int(x), idd))    #result single digital number, key
# step1=[[i for i in data if int(i[0])==j] for j in dig]
# comb=np.asarray([[i for i in itertools.combinations(j,2)] for j in step1])
# #print comb

# name=[[[float(i[0]) for i in j] for j in k] for k in comb]
# flux=[[[float(i[1]) for i in j] for j in k] for k in comb]
# fluxerr=[[[float(i[1]) for i in j] for j in k] for k in comb]
# flux_ratio=[[j[0]/j[1] for j in i] for i in flux]
# fluxerr_ratio=[[fratio((j[0])[1],(j[1])[1],(j[0])[2],(j[1])[2]) for j in i]for i in comb]
# #print fluxerr_ratio
# name_ratio=[['%s/%s'%(i[0],i[1]) for i in a] for a in name]

# data2=np.asarray(zip(name_ratio,flux_ratio))
# #print data2

################## 1.1/4.1/6.1/9.1 ... ###############

key2=[j for j in itertools.combinations(dig,2)]
key3=[set([dex(j) for j in i[:,0]]) for i in comb2]  #store set of first decimal place in each box
#print key3
comb2=np.asarray([[j for j in data if (int(j[0])==i[0] or int(j[0])==i[1])] for i in key2])  #bulk combination
comb3=[];
# print comb2
for x in zip(comb2,key3):
    comb3.append([[i for i in x[0] if dex(i[0])==kk] for kk in x[1]])
# print np.asarray(comb3)
name_ratio= decomp([['%s/%s'%((j[0])[0],(j[1])[0])  for j in i] for i in comb3])
flux_ratio= decomp([[(j[0])[1]/(j[1])[1]  for j in i] for i in comb3])
fluxerr_ratio=decomp([[fratio((j[0])[1],(j[1])[1],(j[0])[2],(j[1])[2])  for j in i] for i in comb3])
# print fluxerr_ratio

[[  1.1         16.86397695   2.0960064 ]
 [  1.2         20.76251695   2.61524173]
 [  1.3         23.49416422   3.52290354]
 [  4.1          8.08499955   1.6380783 ]
 [  4.2          9.33598188   1.81439095]
 [  4.3          7.60256252   1.99328261]
 [  6.1         49.00044228   5.1453253 ]
 [  6.2         56.07896634   4.77828184]
 [  6.3         57.22687092   6.26889785]
 [  9.1          2.01595113   0.57012231]
 [  9.2          2.08065464   0.65703477]
 [  9.3          1.41370893   0.5325526 ]]


In [433]:
for i in range(0,12,3):
    print (data[i,1]+data[i+1,1]+data[i+2,1])/1.

61.1206581187
25.0235439413
162.306279539
5.51031469962


In [308]:
# import matplotlib.pyplot as plt
# name_r=[]; flux_r=[]; fluxerr_r=[]
# [[name_r.append(j) for j in i] for i in name_ratio]
# [[flux_r.append(j) for j in i] for i in flux_ratio]
# [[fluxerr_r.append(j) for j in i] for i in fluxerr_ratio]
# mag1=[7.90174,9.07285,10.7977]
# mag4=[4.18165,5.99801,3.71509]
# mag6=[4.19941,5.536,6.47561]
# mag9=[2.41547,5.20147,5.02835]

# n=len(name_r)

# def rat(x):
#     r=[0.]*3
#     r[0]=x[0]/x[1]
#     r[1]=x[0]/x[2]
#     r[2]=x[1]/x[2]
#     return r
# mag_r=[]
# for i in rat(mag1),rat(mag4),rat(mag6),rat(mag9):
#     for j in i: mag_r.append(j)

# flux_mag= np.asarray(flux_r)/np.asarray(mag_r)

# ax = plt.subplot(111)
# plt.errorbar(range(n),flux_r,fmt='o',yerr=fluxerr_r,label='flux',c='b')
# plt.plot(range(n),mag_r,'o',label='magnitude',c='g')
# plt.plot(range(n),flux_mag,'o',label='flux/mag',c='r')
# for i in range(n):
#     plt.text(i-0.25,0.3,name_r[i],fontsize=16)
# plt.plot(range(n),[1]*n)
    
# plt.legend(loc='lower center', shadow=True, bbox_to_anchor=(0.5, -0.05),
#           fancybox=True, ncol=3)
# plt.suptitle('overview',fontsize=20)
# plt.axis([-1, n,0,max(flux_mag)+0.25])
# #ax.set_xticklabels([])
# ax.xaxis.set_visible(False)

# plt.show()


In [447]:
import matplotlib.pyplot as plt
n=len(flux_ratio)
ax = plt.subplot(111)
const=[61.1206581187,25.0235439413,162.306279539,5.51031469962]
const2=[i[0]/i[1] for i in itertools.combinations(const,2)]
const3=[]
for i in const2:
    for j in range(3):
        const3.append(i)
flux_ratio=np.asarray(flux_ratio)

#plt.errorbar(range(n),flux_ratio,fmt='o',yerr=fluxerr_ratio, label='flux',c='b')
plt.plot(range(n),flux_ratio/const3,'o',c='b',label='flux')
for i in range(n):
    plt.text(i-0.25,0.4,name_ratio[i],fontsize=16)
plt.axis([-1, n,0,max(flux_ratio/const3)+1])
plt.legend(loc='lower center', shadow=True, bbox_to_anchor=(0.5, -0.05),
          fancybox=True, ncol=3)
ax.xaxis.set_visible(False)

flux=[ [16.86397695 , 20.76251695 , 23.49416422],  [ 8.08499955 ,  9.33598188
  , 7.60256252] , [49.00044228  ,56.07896634 , 57.22687092] ,  [2.01595113,
   2.08065464 ,  1.41370893]]
plt.show()

# print flux_ratio
# print fluxerr_ratio
# count=0
# for i in flux:
#     if count<3:
#         y.append(i)
a=[sum(i) for i in flux]
# print a


[2.4425260571434757, 2.4425260571434757, 2.4425260571434757, 0.3765760529555699, 0.3765760529555699, 0.3765760529555699, 11.092044910414096, 11.092044910414096, 11.092044910414096, 0.15417483545537855, 0.15417483545537855, 0.15417483545537855, 4.5412186608916665, 4.5412186608916665, 4.5412186608916665, 29.45499275208236, 29.45499275208236, 29.45499275208236]
